In [3]:
#export
import pandas as pd
import numpy as np
import pdvega
import json

In [ ]:
DATA_LOC = '../data/'

## Standard for næringsgruppering (SN)
kilde: https://www.ssb.no/klass/klassifikasjoner/6

In [ ]:
nace_2007 = pd.read_csv(DATA_LOC + '30.csv', sep=';', encoding='iso-8859-1') 
nace_2007.head(1)

## NACE (næringsgruppe) koder pr foretak

In [ ]:
nace = pd.read_csv(DATA_LOC + 'EnhetNacekode.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 
nace_pivot = pd.pivot_table(nace.sort_values(by=['orgnr', 'rekkefolge']),
                            index='orgnr', columns='rekkefolge',
                            values='nacekode', aggfunc='first').reset_index()
nace_pivot.columns=['orgnr', 'nace_1', 'nace_2', 'nace_3','nace_4']
nace_pivot.head(1)

In [ ]:
len(nace_pivot)

In [ ]:
enhet = pd.read_csv(DATA_LOC + 'EnhetNavnOrgform.tab', sep='\t', encoding='iso-8859-1') 
enhet.head()

In [ ]:
len(enhet)

## Beskrivelser for NACE kodene

In [ ]:
nace_beskrivelse = pd.read_csv(DATA_LOC + 'nace_beskrivelse.tab', sep='\t', encoding='utf-8', dtype={'nacekode': object}) 
nace_beskrivelse.drop(['beskrivelse2'], axis=1, inplace=True)
nace_beskrivelse.columns = ['nace', 'tekst']

In [ ]:
nace_beskrivelse.head()

In [ ]:
nace_beskrivelse.to_json(DATA_LOC + 'nace_beskrivelse.json', orient="records")

## Formålet med virksomheten beskrevet i tekst

In [ ]:
formål = pd.read_csv(DATA_LOC + 'EnhetOrgformVirksomhetArtBransje.tab', sep='\t', encoding='iso-8859-1') 
formål.columns = ['orgnr', 'orgform', 'linje_nr', 'linje_tekst']

In [ ]:
len(formål)

In [ ]:
formål.head(5)

In [ ]:
# eksempel: før sammenslåing
list(formål[formål['orgnr'] == 810034882]['linje_tekst'])

In [ ]:
# eksempel: etter sammenslåing
formål_t = formål[['orgnr','linje_tekst']].groupby('orgnr')['linje_tekst'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
list(formål_t[formål_t['orgnr'] == 810034882]['linje_tekst'])

In [ ]:
len(formål_t)

In [ ]:
formål_t.head()

In [ ]:
df = pd.merge(formål_t, nace_pivot, on='orgnr', how='inner') 

In [ ]:
# TODO: drop konkurser, ukjent osv

In [ ]:
len(df)

In [ ]:
df['lengde_linje_tekst'] = df['linje_tekst'].apply(lambda x: len(x))
df = df[df['lengde_linje_tekst']>4]

In [ ]:
len(df)

## Datasett for maskinlæring

In [ ]:
import spacy
import re

# laster spåkmodell fra norsk_spacy (nb! denne er ikke publisert offentlig enda og vil ikke fungere utenfor NAV)
nlp = spacy.load('nb_ud_ner')

def tokenize(x):
    tokens = nlp.tokenizer(x)
    return [t.text.lower() for t in tokens if not t.is_punct]

In [ ]:
# test
tokenize('Handel og innstallasjonsvirksomhet, eller annen virksomhet forbundet med dette, samt delta I annen virksomhet.')

In [ ]:
#export
def get_dataset(row):
    s = row['linje_tekst']
    tokens = tokenize(s)
    tokens.append(f"__label__{row['nace_1']}")
    linje = ' '.join(tokens)
    return linje

In [ ]:
dataset = df.apply(get_dataset, axis=1)

In [ ]:
dataset.head(5)

In [ ]:
len(dataset)

In [ ]:
dataset.iloc[0]

In [ ]:
# oppdeling av datasettet i 90% trening, 10% validering
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.1)
#train, test = np.split(dataset.sample(frac=1), [int(.9*len(dataset))])

In [ ]:
train.to_csv(DATA_LOC + 'næringskoder_train.txt',
                                       header = None,
                                      index = None,
                                      sep = ' ', encoding='utf-8')

In [ ]:
test.to_csv(DATA_LOC + 'næringskoder_test.txt',
                                       header = None,
                                      index = None,
                                      sep = ' ', encoding='utf-8')

## Trene modell 

In [ ]:
#export
import fastText
from fastText import train_supervised, tokenize, load_model, unicode_literals

In [ ]:
# list metoder
# dir(fastText)

In [ ]:
#export
model = train_supervised(
    input = DATA_LOC + 'næringskoder_train.txt',
    wordNgrams = 3,
    label = '__label__',
    verbose = 2,
    minCount = 1,
    neg = 10
)

In [ ]:
# list metoder
# dir(model)

In [ ]:
model.save_model(DATA_LOC + 'nace_model.bin')

In [ ]:
model.quantize(DATA_LOC + 'nace_model_quantized.bin')

In [ ]:
#export
def print_results(N,p,r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1,p))
    print("R@{}\t{:.3f}".format(1,r))

In [ ]:
print_results(*model.test(DATA_LOC + 'næringskoder_validate.txt'))

In [ ]:
#export
def predict_nace(model, tekst, k):
    res = {}
    res['nace'] = []
    pred = model.predict(tekst, k=k)
    for index, item in enumerate(pred[0]):
        res['nace'].append({'nace': item.replace('__label__', ''), 'prob': pred[1][index] })
    return res

In [ ]:
tekst = test.iloc[400][:-16]
print(tekst)

In [ ]:
predict_nace(model, tekst, 5)

In [ ]:
test.values

In [ ]:
test.values[:10]

In [ ]:
pred_list=[]
for index, row in enumerate(test.values):
    nace = row[-6:]
    line = row[:-16]
    length = len(line)
    if (length > 1):
        pred = predict_nace(model, line,3)
        pred_1 = pred['nace'][0]['nace'][-7:-1]
        pred_2 = pred['nace'][1]['nace'][-7:-1]
        pred_3 = pred['nace'][2]['nace'][-7:-1]

        korrekt = 'Nei'
        if pred_1 == nace:
            korrekt = 'Ja'

        if nace in [pred_2,pred_3]:
            korrekt = 'Delvis'

        pred_list.append([index, nace, korrekt, pred_1, pred_2, pred_3,line])
    else:
        print(row)
        
pred = pd.DataFrame(pred_list)
pred.columns=['index','nace','korrekt','pred_1','pred_2','pred_3','tekst']

In [ ]:
# 10% split
len(pred)

In [ ]:
pred.head()

In [ ]:
resultat = pred[['index','nace','korrekt']].groupby(['nace','korrekt']).count().reset_index().rename(columns={'index':'count'})

In [ ]:
resultat.head()

In [ ]:
pv = pd.pivot_table(resultat, index='nace', columns='korrekt').reset_index()

In [ ]:
pv = pd.pivot_table(resultat, index='nace', columns='korrekt').reset_index()
pv.fillna(0, inplace = True)
pv.columns = ['nace', 'delvis_korrekt','korrekt','feil']
pv['antall'] = pv['feil'] + pv['korrekt'] + pv['delvis_korrekt']
pv['andel_korrekt'] = 100 * pv['korrekt'] / pv['antall']
pv['andel_feil'] = 100 * pv['feil'] / pv['antall']
pv['andel_delvis_korrekt'] = 100 * (pv['korrekt'] + pv['delvis_korrekt']) / pv['antall']
pv.sort_values(by='andel_feil', ascending = False, inplace=True) 

In [ ]:
count = resultat[['nace','count']].groupby('nace').sum().reset_index()
pv = pd.merge(pv, count, how='left', on='nace')

## Andel korrekte fastsettelser

In [ ]:
pv_t = pd.merge(pv,nace_beskrivelse, how='left', on='nace')
pv_t.rename(columns={'tekst':'nace_beskrivelse'}, inplace=True)

### Høy treffsikkerhet

In [ ]:
pv_t.sort_values(by='andel_feil', ascending = True, inplace=True)
pv_t.head(20)

### Lav treffsikkerhet

In [ ]:
pv_t.tail(20)

In [ ]:
plot = pv_t.vgplot.scatter(x='antall', y='andel_korrekt', c='andel_delvis_korrekt')
plot.display()

In [ ]:
#plot.spec

In [ ]:
pv_t.vgplot.scatter(x='antall', y='andel_delvis_korrekt', c='andel_korrekt')

In [ ]:
pv_t['sum_antall'] = pv_t['antall'].cumsum()

In [ ]:
pv_t.vgplot.scatter(x='andel_feil', y='sum_antall', c='andel_korrekt')

In [ ]:
pv_t['nace_hoved'] = pv_t['nace'].apply(lambda x: x[0:2])

In [ ]:
pv_t.head(1)

In [ ]:
pvh = pv_t[['nace_hoved','antall','korrekt','delvis_korrekt','feil']].groupby(['nace_hoved']).sum().reset_index()
pvh = pvh[pvh['antall']>0]
pvh['andel_feil'] = 100*pvh['feil']/pvh['antall']

In [ ]:
pvh = pd.merge(pvh, nace_2007[['code','name']], how="left", left_on='nace_hoved', right_on='code')
pvh.rename(columns={'name':'nace_hoved_beskrivelse'}, inplace=True)

In [ ]:
pvh.sort_values(by='andel_feil', ascending = True, inplace=True) 

In [ ]:
pvh.head(20)

In [ ]:
pvh.tail(20)

In [ ]:
pvh['sum_antall'] = pvh['antall'].cumsum()

In [ ]:
pvh.vgplot.scatter(x='andel_feil', y='sum_antall')

In [ ]:
pred['nace_hoved'] = pred['nace'].apply(lambda x: x[0:2])
data = pd.merge(pred,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', on='nace')
data.head(10)

In [ ]:
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_1', right_on='nace', suffixes=['','_pred_1'])
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_2', right_on='nace', suffixes=['','_pred_2'])
data = pd.merge(data,nace_beskrivelse.rename(columns={'tekst':'nace_beskrivelse'}), how='left', left_on='pred_3', right_on='nace', suffixes=['','_pred_3'])

In [ ]:
from IPython.display import display
display(data)

In [ ]:
print(f"Antall: {len(data[data['nace']=='46.150'])}")
data[data['nace_hoved']=='13']

In [ ]:
data[data['nace_hoved']=='61']

In [ ]:
data[data['nace_hoved']=='06']

In [ ]:
print(f"Antall: {len(data[data['nace']=='94.991'])}")
data[data['nace']=='94.991'].head(20)

In [ ]:
writer = pd.ExcelWriter(DATA_LOC + 'prediksjon.xlsx')
data.to_excel(writer,'prediksjon')
writer.save()